In this homework, we will use Credit Card Data from [the previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/04-evaluation/homework.md).

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv

--2023-04-06 21:46:04--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73250 (72K) [text/plain]
Saving to: ‘AER_credit_card_data.csv’

AER_credit_card_dat 100%[===================>]  71,53K  --.-KB/s    in 0,01s   

2023-04-06 21:46:04 (6,48 MB/s) - ‘AER_credit_card_data.csv’ saved [73250/73250]



In [2]:
!pip install pipenv

In [3]:
!pipenv --version

pipenv, version 2022.12.19



## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

### $Ans = pipenv,\ version\ 2022.12.19 $

## Question 2

* Use Pipenv to install Scikit-Learn version 1.0.2
* What's the first hash for scikit-learn you get in Pipfile.lock?

Ans: `"sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b"`

Note: you should create an empty folder for homework
and do it there. 


## Models

We've prepared a dictionary vectorizer and a model.

They were trained (roughly) using this code:

```python
features = ['reports', 'share', 'expenditure', 'owner']
dicts = df[features].to_dict(orient='records')

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(dicts)

model = LogisticRegression(solver='liblinear').fit(X, y)
```

> **Note**: You don't need to train the model. This code is just for your reference.

And then saved with Pickle. Download them:

* [DictVectorizer](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin?raw=true)
* [LogisticRegression](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin?raw=true)

With `wget`:

```bash
PREFIX=https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework
wget $PREFIX/model1.bin
wget $PREFIX/dv.bin
```

In [18]:
!pipenv install scikit-learn==1.0.2 numpy flask gunicorn

Installing scikit-learn==1.0.2...
⠙ ✔ Installation Succeeded....
Installing numpy...
⠹ ✔ Installation Succeeded
Installing flask...
⠙ ✔ Installation Succeeded
Installing gunicorn...
⠙ ✔ Installation Succeeded
Pipfile.lock (9169a5) out of date, updating to (527572)...
Locking [packages] dependencies...
⠸ ✔ Success!dependencies.....
Locking [dev-packages] dependencies...
Updated Pipfile.lock (01dd6cf04d65f924b1611a6e26d19a8438a0fe3e9db3224413ad62cec4527572)!
Installing dependencies from Pipfile.lock (527572)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


In [10]:
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin
!wget https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin

--2023-04-06 22:07:26--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/model1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889 [application/octet-stream]
Saving to: ‘model1.bin’

model1.bin          100%[===================>]     889  --.-KB/s    in 0s      

2023-04-06 22:07:26 (44,6 MB/s) - ‘model1.bin’ saved [889/889]

--2023-04-06 22:07:26--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/course-zoomcamp/cohorts/2022/05-deployment/homework/dv.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.1

In [1]:
import sklearn
print(sklearn.__version__)

1.0.2


## Question 3

Let's use these models!

* Write a script for loading these models with pickle
* Score this client:

```json
{"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
```

What's the probability that this client will get a credit card? 

* 0.162
* 0.391
* 0.601
* 0.993

### $Ans = 0.162 $

If you're getting errors when unpickling the files, check their checksum:

```bash
$ md5sum model1.bin dv.bin
3f57f3ebfdf57a9e1368dcd0f28a4a14  model1.bin
6b7cded86a52af7e81859647fa3a5c2e  dv.bin
```

In [2]:
import pickle

In [3]:
input_file = 'model1.bin'

with open(input_file, 'rb') as f_in: 
    model = pickle.load(f_in)

input_file = 'dv.bin'

with open(input_file, 'rb') as f_in: 
    dv = pickle.load(f_in)

In [4]:
customer = {"reports": 0, "share": 0.001694, "expenditure": 0.12, "owner": "yes"}
X = dv.transform([customer])
y_pred = model.predict_proba(X)
y_pred[0, 1]

0.16213414434326598

## Question 4

Now let's serve this model as a web service

* Install Flask and gunicorn (or waitress, if you're on Windows)
* Write Flask code for serving the model
* Now score this client using `requests`:

```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit card?

* 0.274
* 0.484
* 0.698
* 0.928

### $Ans = 0.928 $

## Docker

Install [Docker](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/05-deployment/06-docker.md). We will use it for the next two questions.

For these questions, we prepared a base image: `svizor/zoomcamp-model:3.9.12-slim`. 
You'll need to use it (see Question 5 for an example).

This image is based on `python:3.9.12-slim` and has a logistic regression model 
(a different one) as well a dictionary vectorizer inside. 

This is how the Dockerfile for this image looks like:

```docker 
FROM python:3.9.12-slim
WORKDIR /app
COPY ["model2.bin", "dv.bin", "./"]
```

We already built it and then pushed it to [`svizor/zoomcamp-model:3.9.12-slim`](https://hub.docker.com/r/svizor/zoomcamp-model).

> **Note**: You don't need to build this docker image, it's just for your reference.


In [5]:
import flask
flask.__version__

'2.2.2'

In [6]:
import gunicorn
print(gunicorn.__version__)

20.1.0


In [7]:
import requests

url = 'http://localhost:9696/predict'
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
print(requests.post(url, json=client).json())

{'churn': True, 'churn_probability': 0.9282218018527452}


## Question 5

Download the base image `svizor/zoomcamp-model:3.9.12-slim`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 15 Mb
* 125 Mb
* 275 Mb
* 415 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

### $Ans = 125 Mb $

## Dockerfile

Now create your own Dockerfile based on the image we prepared.

It should start like that:

```docker
FROM svizor/zoomcamp-model:3.9.12-slim
# add your stuff here
```

Now complete it:

* Install all the dependencies form the Pipenv file
* Copy your Flask script
* Run it with Gunicorn 

After that, you can build your docker image.

In [9]:
!docker pull svizor/zoomcamp-model:3.9.12-slim

3.9.12-slim: Pulling from svizor/zoomcamp-model

a5fb9032: Pulling fs layer 
81b69b9a: Pulling fs layer 
a0a62ab5: Pulling fs layer 
28511179: Pulling fs layer 
79bc1c47: Pulling fs layer 
c3f7336a: Pulling fs layer 
Digest: sha256:10445b40653d5ac17ede84db17f42ae8c4090b347a979372b8102174498b33b9[7A
Status: Downloaded newer image for svizor/zoomcamp-model:3.9.12-slim
docker.io/svizor/zoomcamp-model:3.9.12-slim


In [10]:
!docker images

REPOSITORY                    TAG           IMAGE ID       CREATED         SIZE
zoomcamp-test                 latest        8adb703c5d97   9 hours ago     161MB
taxi_ingest                   v001          35e3f439dfcc   2 months ago    996MB
<none>                        <none>        d25939a74462   2 months ago    996MB
<none>                        <none>        48465b76275b   2 months ago    996MB
<none>                        <none>        0cae90a7774d   2 months ago    996MB
<none>                        <none>        3ce1121e827c   2 months ago    996MB
<none>                        <none>        24f8c8b3dec0   2 months ago    996MB
manojkl/prefect               zoom          6f30d5a867f8   2 months ago    885MB
<none>                        <none>        fe83f6b4ae6a   2 months ago    885MB
manojkl/prefect               <none>        7b992cdbb0d1   2 months ago    885MB
<none>                        <none>        7d1c31e4b092   2 months ago    996MB
test                         

In [24]:
!docker build -t zoomcamp-hw5 .

[+] Building 0.0s (0/1)                                                         
[+] Building 0.2s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 301B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for docker.io/svizor/zoomcamp-model:3.9.12-s  0.1s
[+] Building 0.3s (2/3)                                                         
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 301B                                       0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => [internal] load metadata for dock

In [12]:
!docker scan

Docker Scan relies upon access to Snyk, a third party provider, do you consent to proceed using Snyk? (y/N)
^C


## Question 6

Let's run your docker container!

After running it, score this client once again:

```python
url = "YOUR_URL"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()
```

What's the probability that this client will get a credit card now?

* 0.289
* 0.502
* 0.769
* 0.972

### $Ans = 0.769 $

In [20]:
!docker run -it --rm -p 9696:9696 zoomcamp-hw5

[2023-04-06 21:18:24 +0000] [1] [INFO] Starting gunicorn 20.1.0
[2023-04-06 21:18:24 +0000] [1] [INFO] Listening at: http://0.0.0.0:9696 (1)
[2023-04-06 21:18:24 +0000] [1] [INFO] Using worker: sync
[2023-04-06 21:18:24 +0000] [9] [INFO] Booting worker with pid: 9
^C
[2023-04-06 21:19:05 +0000] [1] [INFO] Handling signal: int
[2023-04-06 21:19:05 +0000] [9] [INFO] Worker exiting (pid: 9)


In [25]:
url = "http://localhost:9696/predict"
client = {"reports": 0, "share": 0.245, "expenditure": 3.438, "owner": "yes"}
requests.post(url, json=client).json()

{'churn': True, 'churn_probability': 0.7692649226628628}